In [563]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch_geometric.nn import GCNConv, GINConv, GINEConv, GraphConv, GENConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import global_mean_pool, global_max_pool, global_add_pool, global_sort_pool

from all_2_states import incidence_matrix, all_goal_states_ls, all_nongoal_states_ls, all_goal_states_np, all_nongoal_states_np, edge_tps_np


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [587]:
for state in all_goal_states_ls:
    print(state)
    
print(100*'=')

for state in all_nongoal_states_ls:
    print(state)

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5]
[2, 2, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 0, 0, 0, 0, 4, 4, 4, 4, 5, 5, 5, 5]
[1, 1, 1, 1, 0, 0, 0, 0, 3, 3, 3, 3, 2, 2, 2, 2, 4, 4, 4, 4, 5, 5, 5, 5]
[3, 3, 3, 3, 2, 2, 2, 2, 0, 0, 0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 5, 5, 5, 5]
[0, 0, 0, 0, 1, 1, 1, 1, 3, 3, 3, 3, 2, 2, 2, 2, 5, 5, 5, 5, 4, 4, 4, 4]
[3, 3, 3, 3, 2, 2, 2, 2, 1, 1, 1, 1, 0, 0, 0, 0, 5, 5, 5, 5, 4, 4, 4, 4]
[1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 3, 5, 5, 5, 5, 4, 4, 4, 4]
[2, 2, 2, 2, 3, 3, 3, 3, 0, 0, 0, 0, 1, 1, 1, 1, 5, 5, 5, 5, 4, 4, 4, 4]
[0, 0, 0, 0, 1, 1, 1, 1, 5, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 3, 3]
[5, 5, 5, 5, 4, 4, 4, 4, 1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 3, 3, 3, 3]
[1, 1, 1, 1, 0, 0, 0, 0, 4, 4, 4, 4, 5, 5, 5, 5, 2, 2, 2, 2, 3, 3, 3, 3]
[4, 4, 4, 4, 5, 5, 5, 5, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]
[0, 0, 0, 0, 1, 1, 1, 1, 4, 4, 4, 4, 5, 5, 5, 5, 3, 3, 3, 3, 2, 2, 2, 2]
[4, 4, 4, 4, 5, 5, 5, 5, 1, 1, 1, 1, 0, 0, 0, 0, 3,

In [588]:
# edge_tps_torch = F.one_hot(torch.tensor(edge_tps_np).long(), 3)
# print(edge_tps_np)
# print(np.repeat(np.expand_dims(edge_tps_np, axis=1), 6, axis=1))
edge_tps_torch = torch.tensor(np.repeat(np.expand_dims(edge_tps_np, axis=1), 6, axis=1), dtype=torch.long)
# edge_tps_torch = torch.tensor(edge_tps_np, dtype=torch.long)

dataset = []

dataset_fc = []
y_fc = []

for x in all_goal_states_ls:
    features = torch.tensor(x, dtype=torch.long)
    features = F.one_hot(features, 6)
    dataset_fc.append(features.detach().cpu().numpy())
    y_fc.append(1)
    edge_index = torch.tensor(incidence_matrix, dtype=torch.long)
    print(edge_tps_torch.type())
    data_obj = Data(x=features, edge_index=edge_index, edge_attr=edge_tps_torch, y=1)
    dataset.append(data_obj)


for x in all_nongoal_states_ls:
    features = torch.tensor(x, dtype=torch.long)
    features = F.one_hot(features, 6)
    dataset_fc.append(features.detach().cpu().numpy())
    y_fc.append(0)
    edge_index = torch.tensor(incidence_matrix, dtype=torch.long)
    data_obj = Data(x=features, edge_index=edge_index, edge_attr=edge_tps_torch, y=0)
    dataset.append(data_obj)

print(len(dataset))
print(dataset[0])

train_loader = DataLoader(dataset, batch_size=48, shuffle=False)

In [675]:
dataset_2 = dataset[:12] + dataset[24:36]
dataset_fc2 = dataset_fc[:12] + dataset_fc[24:36]
y_fc2 = y_fc[:12] + y_fc[24:36]

half_loader = DataLoader(dataset_2, batch_size=24, shuffle=False)

In [594]:
class FCModel(nn.Module):
    def __init__(self, state_dim: int, one_hot_depth: int, fc1_dim: int, fc2_dim: int):
        super().__init__()
        self.one_hot_depth: int = one_hot_depth
        self.state_dim: int = state_dim

        self.fc1 = nn.Linear(self.state_dim * self.one_hot_depth, fc1_dim)
        self.fc2 = nn.Linear(fc1_dim, fc2_dim)
        self.fc_out = nn.Linear(fc2_dim, 1)

    def forward(self, states_nnet, training=False):
        x = states_nnet

        # preprocess input
        x = torch.tensor(x).float()
        x = x.view(-1, self.state_dim * self.one_hot_depth)

        # first two hidden layers
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        # output
        x = self.fc_out(x)
        x = F.sigmoid(x)
        return x

In [658]:
class GNModel(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNModel, self).__init__()
        # torch.manual_seed(12345)
#         self.conv1 = GINEConv(nn.Linear(6, hidden_channels))
#         self.conv2 = GINEConv(nn.Linear(hidden_channels, hidden_channels))
#         self.conv3 = GINEConv(nn.Linear(hidden_channels, hidden_channels))
#         self.conv1 = GraphConv(6, hidden_channels)
#         self.conv2 = GraphConv(hidden_channels, hidden_channels)
#         self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.conv1 = GENConv(6, hidden_channels, aggr='softmax', t=1.0, learn_t=True, num_layers=2, norm='layer')
        self.conv2 = GENConv(hidden_channels, hidden_channels, aggr='softmax', t=1.0, learn_t=True, num_layers=2, norm='layer')
        self.conv3 = GENConv(hidden_channels, hidden_channels, aggr='softmax', t=1.0, learn_t=True, num_layers=2, norm='layer')
        self.lin = nn.Linear(hidden_channels, 1)

    def forward(self, x, edge_index, edge_attr, batch):
        # 1. Obtain node embeddings
#         print(x[0].size(-1))
#         print(edge_attr.size(-1))
        assert edge_attr.size(-1) == x[0].size(-1)
#         print(edge_attr)
#         print(edge_attr.type())
        x = self.conv1(x.float(), edge_index=edge_index, edge_attr=edge_attr)
        x = F.relu(x)
        edge_attr = torch.tensor(np.repeat(edge_attr, 4, axis=1), dtype=torch.long)
#         print(x[0].size(-1))
#         print(edge_attr.size(-1))
        x = self.conv2(x, edge_index=edge_index, edge_attr=edge_attr)
#         x = F.relu(x)
#         x = self.conv3(x, edge_index=edge_index, edge_attr=edge_attr)

        # 2. Readout layer
        x = global_add_pool(x, batch)  # [batch_size, hidden_channels]
#         print(x.size())
#         print(x[0:24])
# #         print(x[1])
#         print(x[24:])
#         print(x[25])

        # 3. Apply a final classifier
        # x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        x = F.sigmoid(x)
        return x

In [672]:
model = GNModel(hidden_channels=24)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

In [673]:
def run_train(loader):
    model.train()

    for data in loader:  # Iterate in batches over the training dataset.
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)  # Perform a single forward pass.
#         print(out.squeeze())
#         print(data.y)
        print(data.y.type())
        loss = criterion(out.squeeze(), data.y.float())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    print(loss.item())

def run_test(loader):
    model.eval()
    misclass = []

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
#         pred = out.argmax(dim=1)  # Use the class with highest probability.
        result = []
        for output in out:
            if output > 0.5:
                result.append(1)
            else:
                result.append(0)
        pred = torch.tensor(result)
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        misclass += list(data[pred != data.y])
    return correct / len(loader.dataset), misclass  # Derive ratio of correct predictions.

print(train_loader)

In [674]:
for epoch in range(1, 100):
    run_train(train_loader)
    train_acc, trM = run_test(train_loader)
#     print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
    if train_acc == 1.0:
        break

<ipython-input-658-79d9e753907c>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(np.repeat(edge_attr, 4, axis=1), dtype=torch.long)


torch.LongTensor
4.718338489532471
Epoch: 001, Train Acc: 0.5000
torch.LongTensor
3.544243097305298
Epoch: 002, Train Acc: 0.5000
torch.LongTensor
2.392704486846924
Epoch: 003, Train Acc: 0.5000
torch.LongTensor
1.3189719915390015
Epoch: 004, Train Acc: 0.5000
torch.LongTensor
0.6886625289916992
Epoch: 005, Train Acc: 0.5000
torch.LongTensor
0.993502676486969
Epoch: 006, Train Acc: 0.5000
torch.LongTensor
1.4493776559829712
Epoch: 007, Train Acc: 0.5000
torch.LongTensor
1.6396570205688477
Epoch: 008, Train Acc: 0.5000
torch.LongTensor
1.5916765928268433
Epoch: 009, Train Acc: 0.5000
torch.LongTensor
1.382279872894287
Epoch: 010, Train Acc: 0.5000
torch.LongTensor
1.077668309211731
Epoch: 011, Train Acc: 0.5000
torch.LongTensor
0.7963201403617859
Epoch: 012, Train Acc: 1.0000


In [662]:
model2 = FCModel(24, 6, 24, 24)
optimizer = torch.optim.Adam(model2.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

dataset_fc = torch.tensor(dataset_fc, dtype=torch.long)
y_fc = torch.tensor(y_fc, dtype=torch.float32)

<ipython-input-662-aaa2af9e224a>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_fc = torch.tensor(dataset_fc, dtype=torch.long)
<ipython-input-662-aaa2af9e224a>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_fc = torch.tensor(y_fc, dtype=torch.float32)


In [653]:
def run_fc_train():
    model2.train()

    out = model2(dataset_fc)  # Perform a single forward pass.\
    print(len(out))
    print(len(y_fc))
    loss = criterion(out.squeeze(), y_fc)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    optimizer.zero_grad()  # Clear gradients.
    print(loss.item())

def run_fc_test():
    model2.eval()
    misclass = []

    correct = 0
    out = model2(dataset_fc)
#     pred = out.argmax(dim=1)  # Use the class with highest probability.
    result = []
    for output in out:
        if output > 0.5:
            result.append(1)
        else:
            result.append(0)
    pred = torch.tensor(result)
    correct += int((pred == y_fc).sum())  # Check against ground-truth labels.
    misclass += list(dataset_fc[pred != y_fc])
    return correct / len(dataset_fc), misclass  # Derive ratio of correct predictions.



In [654]:
for epoch in range(1, 101):
    run_fc_train()
    train_acc, trM = run_fc_test()
#     print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')

<ipython-input-594-e4f1a6919e56>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).float()


48
48
0.69993656873703
Epoch: 001, Train Acc: 0.5000
48
48
0.6951406002044678
Epoch: 002, Train Acc: 0.5000
48
48
0.6905686259269714
Epoch: 003, Train Acc: 0.5000
48
48
0.6861886382102966
Epoch: 004, Train Acc: 0.5000
48
48
0.6819456219673157
Epoch: 005, Train Acc: 0.5000
48
48
0.6777820587158203
Epoch: 006, Train Acc: 0.5000
48
48
0.6736657023429871
Epoch: 007, Train Acc: 0.5000
48
48
0.6695994734764099
Epoch: 008, Train Acc: 0.5000
48
48
0.6654258370399475
Epoch: 009, Train Acc: 0.5000
48
48
0.6609951853752136
Epoch: 010, Train Acc: 0.5000
48
48
0.6562570929527283
Epoch: 011, Train Acc: 0.5000
48
48
0.651249349117279
Epoch: 012, Train Acc: 0.5000
48
48
0.646018385887146
Epoch: 013, Train Acc: 0.5000
48
48
0.6404486894607544
Epoch: 014, Train Acc: 0.5625
48
48
0.6344786286354065
Epoch: 015, Train Acc: 0.5833
48
48
0.6280642151832581
Epoch: 016, Train Acc: 0.6458
48
48
0.6213086843490601
Epoch: 017, Train Acc: 0.7083
48
48
0.6142169833183289
Epoch: 018, Train Acc: 0.7292
48
48
0.606813

------

In [676]:
half_model = GNModel(hidden_channels=24)
optimizer = torch.optim.Adam(half_model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

In [677]:
def run_train2(loader):
    half_model.train()

    for data in loader:  # Iterate in batches over the training dataset.
        out = half_model(data.x, data.edge_index, data.edge_attr, data.batch)  # Perform a single forward pass.
#         print(out.squeeze())
#         print(data.y)
        print(data.y.type())
        loss = criterion(out.squeeze(), data.y.float())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
    print(loss.item())

def run_test2(loader):
    half_model.eval()
    misclass = []

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = half_model(data.x, data.edge_index, data.edge_attr, data.batch)
#         pred = out.argmax(dim=1)  # Use the class with highest probability.
        result = []
        for output in out:
            if output > 0.5:
                result.append(1)
            else:
                result.append(0)
        pred = torch.tensor(result)
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        misclass += list(data[pred != data.y])
    return correct / len(loader.dataset), misclass  # Derive ratio of correct predictions.

print(train_loader)

In [678]:
for epoch in range(1, 100):
    run_train2(half_loader)
    train_acc, trM = run_test2(train_loader)
#     print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
    if epoch % 1 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
    if train_acc == 1.0:
        break

<ipython-input-658-79d9e753907c>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_attr = torch.tensor(np.repeat(edge_attr, 4, axis=1), dtype=torch.long)
C:\Users\marti\Anaconda3\envs\bakalarka\lib\site-packages\torch\nn\functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


torch.LongTensor
5.825967788696289
Epoch: 001, Train Acc: 0.5000
torch.LongTensor
4.74454927444458
Epoch: 002, Train Acc: 0.5000
torch.LongTensor
3.6753222942352295
Epoch: 003, Train Acc: 0.5000
torch.LongTensor
2.621415376663208
Epoch: 004, Train Acc: 0.5000
torch.LongTensor
1.5980180501937866
Epoch: 005, Train Acc: 0.5000
torch.LongTensor
0.8157267570495605
Epoch: 006, Train Acc: 0.5000
torch.LongTensor
0.8089656829833984
Epoch: 007, Train Acc: 0.5000
torch.LongTensor
1.2864556312561035
Epoch: 008, Train Acc: 0.5000
torch.LongTensor
1.6204530000686646
Epoch: 009, Train Acc: 0.5000
torch.LongTensor
1.7289608716964722
Epoch: 010, Train Acc: 0.5000
torch.LongTensor
1.648837685585022
Epoch: 011, Train Acc: 0.5000
torch.LongTensor
1.4391480684280396
Epoch: 012, Train Acc: 0.5000
torch.LongTensor
1.1575590372085571
Epoch: 013, Train Acc: 0.5000
torch.LongTensor
0.8795127272605896
Epoch: 014, Train Acc: 0.5000
torch.LongTensor
0.7084309458732605
Epoch: 015, Train Acc: 0.5000
torch.LongTenso

In [691]:
half_model2 = FCModel(24, 6, 24, 24)
optimizer = torch.optim.Adam(half_model2.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()

dataset_fc2 = torch.tensor(dataset_fc2, dtype=torch.long)
y_fc2 = torch.tensor(y_fc2, dtype=torch.float32)

<ipython-input-691-75d2b1c6678d>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset_fc2 = torch.tensor(dataset_fc2, dtype=torch.long)
<ipython-input-691-75d2b1c6678d>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_fc2 = torch.tensor(y_fc2, dtype=torch.float32)


In [692]:
def run_fc_train2():
    half_model2.train()

    out = half_model2(dataset_fc2)  # Perform a single forward pass.\
    print(len(out))
    print(len(y_fc))
    loss = criterion(out.squeeze(), y_fc2)  # Compute the loss.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.
    optimizer.zero_grad()  # Clear gradients.
    print(loss.item())

def run_fc_test2(dataset, labels):
    half_model2.eval()
    misclass = []

    correct = 0
    out = half_model2(dataset)
#     pred = out.argmax(dim=1)  # Use the class with highest probability.
    result = []
    for output in out:
        if output > 0.5:
            result.append(1)
        else:
            result.append(0)
    pred = torch.tensor(result)
    correct += int((pred == labels).sum())  # Check against ground-truth labels.
    misclass += list(dataset[pred != labels])
    return correct / len(dataset), misclass  # Derive ratio of correct predictions.



In [693]:
for epoch in range(1, 201):
    run_fc_train2()
    train_acc, trM = run_fc_test2(dataset_fc2, y_fc2)
    test_acc, testM = run_fc_test2(dataset_fc, y_fc)
#     print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}')
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<ipython-input-594-e4f1a6919e56>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x).float()


12
48
0.6614223122596741
12
48
0.6481271386146545
12
48
0.6353236436843872
12
48
0.6222890019416809
12
48
0.6090385913848877
12
48
0.5947416424751282
12
48
0.5800952911376953
12
48
0.5652318596839905
12
48
0.5496399998664856
12
48
0.5332184433937073
Epoch: 010, Train Acc: 1.0000, Test Acc: 0.5000
12
48
0.5159369707107544
12
48
0.4978337585926056
12
48
0.4790693521499634
12
48
0.45961353182792664
12
48
0.43954285979270935
12
48
0.41893115639686584
12
48
0.39783307909965515
12
48
0.3762528598308563
12
48
0.3543708324432373
12
48
0.3322574198246002
Epoch: 020, Train Acc: 1.0000, Test Acc: 0.5000
12
48
0.30979660153388977
12
48
0.28721755743026733
12
48
0.2647142708301544
12
48
0.2423819899559021
12
48
0.22067975997924805
12
48
0.19975578784942627
12
48
0.17981944978237152
12
48
0.16109125316143036
12
48
0.1435948759317398
12
48
0.12747400999069214
Epoch: 030, Train Acc: 1.0000, Test Acc: 0.5000
12
48
0.11266370862722397
12
48
0.09926370531320572
12
48
0.08721210807561874
12
48
0.076403237

In [688]:
trM

[tensor([[1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1]]),
 tensor([[0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 1],
         [1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0],
         [0,